## IMPORT AND PATHS

In [31]:
import warnings
import os
import sys
import random
import math
import glob
import numpy as np
#import skimage.io
import imageio
import cv2
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import imageio
from pathlib import Path
#import imgaug
import tensorflow as tf
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# Root directory of the project

DATASET = 'LUEBECK/'
DIR = Path(os.path.dirname(os.path.realpath('__file__')))


if (DIR == Path('/beegfs/home/users/t/tim.schroeder')):
    CODE_DIR = os.path.join(DIR, 'object_rep/project_code/') 
else: 
    CODE_DIR = DIR

PROJECT_DIR = Path(CODE_DIR).parent

DATA_DIR = os.path.join(PROJECT_DIR, 'project_data/', DATASET)
RESULTS_DIR = os.path.join(PROJECT_DIR, 'project_results/',DATASET)
if os.path.exists(RESULTS_DIR) == False:
    os.mkdir(RESULTS_DIR)

ROOT_DIR = os.path.join(CODE_DIR, "Mask_RCNN-master/")



In [32]:
print('DATASET: %s' % DATASET)
print('DIR: %s' % DIR)
print('CODE_DIR: %s' % CODE_DIR)
print('PROJECT_DIR: %s' % PROJECT_DIR)
print('DATA_DIR: %s' % DATA_DIR)
print('RESULTS_DIR: %s' % RESULTS_DIR)
print('ROOT_DIR: %s' % ROOT_DIR)

DATASET: LUEBECK/
DIR: /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code
CODE_DIR: /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code
PROJECT_DIR: /Users/timschroder/Documents/Uni/Bachelorarbeit
DATA_DIR: /Users/timschroder/Documents/Uni/Bachelorarbeit/project_data/LUEBECK/
RESULTS_DIR: /Users/timschroder/Documents/Uni/Bachelorarbeit/project_results/LUEBECK/
ROOT_DIR: /Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/Mask_RCNN-master/


In [33]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush', 'BG_object_close']

## COCO

In [34]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.visualize import save_image
from mrcnn.visualize import return_image

In [35]:
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
   # BACKBONE = 'resnet101'

config = InferenceConfig()
config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')




Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [36]:
import ntpath
def path_leaf(path): # Get file name from path
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [37]:
path = 'b/b/001.AVI'
path2 = os.path.splitext(path_leaf(path))[0] #Example
os.path.splitext(path2)[0]

'001'

In [38]:
# Load Results when there are non
def save_detect_video(v_name, path, maxframes):
    print('detect_img for video: %s' % v_name)
    detect_img = []
    cap = cv2.VideoCapture(path)
    success,image = cap.read()
    count = 0
    success = True
    while success:
        success,image = cap.read()
        results = model.detect([image], verbose=1)
        detect_img.append(results[0])
        count += 1
    # save to npz file (compressed)
    np.savez_compressed((RESULTS_DIR + '/detect_img_' + v_name), detect_img=detect_img)
    cap.release()
    cv2.destroyAllWindows()



In [39]:
def load_results(PATH, v_name): 
    loadlib = np.load(PATH, allow_pickle=True) 
    results = loadlib['detect_img']
    #np.save(RESULTS_DIR + '/detect_img_' + v_name + '.npy', results)
    return results



In [40]:
PATH = (RESULTS_DIR + f'detect_img_{2:03}.npy') 
os.path.exists(PATH)

False

In [41]:
def read_video(path):
    vidlist = []
    cap = cv2.VideoCapture(path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    if (cap.isOpened()== False): 
        print("Error opening video stream or file: %s" % path)
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            vidlist.append(frame)
        else:
            break 
    
    cap.release()
    cv2.destroyAllWindows()
    return vidlist

## Load Detection Data

In [101]:
v=0
vid_path_list = [video for video in glob.glob(DATA_DIR + '/*')]
vid_path_list.sort()
v_name = os.path.splitext(path_leaf(vid_path_list[v]))[0]
print('Video: %s' % v_name)

vidlist = [] 
vidlist = read_video(vid_path_list[v])
PATH = (RESULTS_DIR + f'detect_data/detect_img_'+ v_name +'.npz') #changed to npy /z 
    detect_data = []
detect_data = load_results(PATH, v_name) #load if they are in RESULT_DIR, otherwise run MRCNN
    #results = np.load(PATH, allow_pickle=True) 
    
    #r = results[i]
    #img = (vidlist[i])
   # write_frame = np.asarr
        
   

In [102]:
len(detect_data)

NameError: name 'gaze_folder_list' is not defined

In [74]:
class_ids = []
for i in range(len(detect_data)):  
    class_ids.append(detect_data[i]['class_ids'])
    
#class_ids = np.asarray(class_ids)

In [100]:
df = pd.DataFrame(class_ids)
df[0].unique()


array([ 1, 15])

In [90]:
df.apply(pd.value_counts)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
1.0,595.0,595.0,593.0,593.0,582.0,569.0,561.0,532.0,513.0,516.0,...,129.0,93.0,75.0,48.0,32.0,26.0,17.0,10.0,6.0,1.0
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13.0,NaN,1.0,1.0,3.0,13.0,27.0,28.0,52.0,78.0,67.0,...,12.0,8.0,3.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN
14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
15.0,2.0,1.0,3.0,NaN,NaN,NaN,NaN,2.0,NaN,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
from collections import Counter
Counter(np.concatenate(class_ids).ravel())

Counter({1: 12645,
         15: 20,
         13: 1068,
         29: 81,
         25: 72,
         26: 70,
         34: 99,
         57: 11,
         18: 39,
         30: 78,
         31: 1,
         36: 25,
         27: 31,
         17: 12,
         2: 34,
         39: 10,
         35: 1,
         7: 1,
         3: 3,
         33: 11,
         40: 1,
         32: 1,
         14: 43,
         38: 3,
         20: 2})

In [82]:
a

Counter({1: 12645,
         15: 20,
         13: 1068,
         29: 81,
         25: 72,
         26: 70,
         34: 99,
         57: 11,
         18: 39,
         30: 78,
         31: 1,
         36: 25,
         27: 31,
         17: 12,
         2: 34,
         39: 10,
         35: 1,
         7: 1,
         3: 3,
         33: 11,
         40: 1,
         32: 1,
         14: 43,
         38: 3,
         20: 2})

## Save Video With Detected Objects


In [16]:
def video_with_os(save_video): #save_video = TRUE: load results, make video; save_video = FALSE make results, save results
    vid_path_list = [video for video in glob.glob(DATA_DIR + '/*')]
    vid_path_list.sort()


    videos_run = [s for s in range(len(vid_path_list))]
    maxframes = 50000
    fps=30
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')

    for v in range(12,17):
        v_name = os.path.splitext(path_leaf(vid_path_list[v]))[0]
        print('Video: %s' % v_name)
        
        vidlist = [] 
        vidlist = read_video(vid_path_list[v])
        
        if save_video == True:
            PATH = (RESULTS_DIR + f'detect_img_'+ v_name +'.npz') #changed to npy /z 
            if os.path.exists(PATH):
                results = []
                results = load_results(PATH, v_name) #load if they are in RESULT_DIR, otherwise run MRCNN
                #results = np.load(PATH, allow_pickle=True) 
                
                frame_width = vidlist[v].shape[1]
                frame_height = vidlist[v].shape[0]

                #Write Video
                out = cv2.VideoWriter(RESULTS_DIR + 'results_' + v_name +'_0.AVI' ,cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width,frame_height))

                for i in range(min(len(vidlist),maxframes)):
                    r = results[i]
                    img = (vidlist[i])
                    write_frame = np.asarray(return_image(img,i,r['rois'], r['masks'], r['class_ids'], r['scores'], class_names))
                    out.write(write_frame)
                
                out.release()
                cv2.destroyAllWindows()
            
            else: 
                print('no detect_img file found for video: %s' % v_name)
                
        else:
            save_detect_video(v_name, vid_path_list[v], maxframes) #run MRCNN



In [17]:
video_with_os(save_video=True)

Video: roundabout
Video: sea
Video: st_petri_gate

*** No instances in image to draw *** 


*** No instances in image to draw *** 


*** No instances in image to draw *** 


*** No instances in image to draw *** 


*** No instances in image to draw *** 

Video: st_petri_market
Video: st_petri_mcdonalds


## Check Results

In [13]:
r = load_results('/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results/LUEBECK/detect_img_roundabout.npz','roundabout')